In [1]:
from entities import *
from constants import *
from simulations import *
from tournament_roster import *
from constants import TEAMS_DATA, STAT_EFFECTS

import pickle
import numpy as np
import pandas as pd
import openpyxl as xl

from pathlib import Path

LOAD = True
PRECOMPUTED_GAMES = Path.cwd() / 'data' / 'pre_computed_games.pkl'
BLANK_BRACKET = Path.cwd() / 'resources' / 'blank_bracket.xlsx'
OUTPUT = Path.cwd() / 'output'

c:\Users\amgal\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

if LOAD:
    with open(PRECOMPUTED_GAMES, 'rb') as f:
        games = pickle.load(f)

In [3]:
wb = xl.load_workbook('resources/blank_bracket.xlsx', data_only=True)

tournament_roster = [teams[wb['Teams'][f'C{i}'].value] for i in range(2, 66)]
tournament_roster

[<Team: Abilene Christian Record: 10-10>,
 <Team: Air Force Record: 3-17>,
 <Team: Akron Record: 17-4>,
 <Team: Alabama Record: 14-6>,
 <Team: Alabama A&M Record: 11-9>,
 <Team: Alabama State Record: 6-14>,
 <Team: Albany (NY) Record: 7-15>,
 <Team: Alcorn State Record: 3-16>,
 <Team: American Record: 13-9>,
 <Team: Appalachian State Record: 14-9>,
 <Team: Arizona Record: 21-0>,
 <Team: Arizona State Record: 11-10>,
 <Team: Arkansas Record: 16-5>,
 <Team: Arkansas State Record: 13-9>,
 <Team: Arkansas-Pine Bluff Record: 7-13>,
 <Team: Army Record: 9-13>,
 <Team: Auburn Record: 14-7>,
 <Team: Austin Peay Record: 14-6>,
 <Team: Ball State Record: 7-13>,
 <Team: Baylor Record: 11-9>,
 <Team: Bellarmine Record: 8-13>,
 <Team: Belmont Record: 19-3>,
 <Team: Bethune-Cookman Record: 9-11>,
 <Team: Binghamton Record: 5-18>,
 <Team: Boise State Record: 13-8>,
 <Team: Boston College Record: 9-11>,
 <Team: Florida Record: 15-6>,
 <Team: Duke Record: 19-1>,
 <Team: Bradley Record: 14-8>,
 <Team: B

In [4]:
def round_of_games(team_list: list, n: int=20, summary=False):
    winners, losers = [], []
    schools = []
    records = []
    analyses = []

    for i in range(0, len(team_list), 2):
        game = simulate_n_games(team_list[i], team_list[i+1], n, summary=summary)
        winners.append(teams[game['Winner']].name)
        losers.append(teams[game['Loser']].name)
        analyses.append(game['Analysis'])
        schools.append([game['Team 1'], game['Team 2']])
        records.append([game['Win Count 1'], game['Win Count 2']])
    
    return {'Winners': winners, 
            'Losers': losers, 
            'Analysis': analyses, 
            'Schools': schools,
            'Records': records
        }

def first_round(region: dict, n: int=20):
    first_round_teams = [teams[v] for v in region.values()]
    return round_of_games(first_round_teams, n=n)

# Precalculated Games

In [5]:
def precomputed_games(team_1, team_2):
    for game in games:
        game_players = [game['Winner'], game['Loser']]
        if (team_1 in game_players) and (team_2 in game_players):
            
            output = {
                'T1': game['Team 1'].name,
                'T1%': game['Team 1 Win Pct'],
                'T2': game['Team 2'].name,
                'T2%': game['Team 2 Win Pct'],
                'W': game['Winner'],
                'Analysis': game['Analysis']
            }

            return output

def precomputed_games_from_list(l):
    for i in range(0, len(l), 2):
        yield precomputed_games(l[i], l[i+1])

def parse_games(outcomes):
    round_t1 = [t['T1'] for t in outcomes]
    round_t2 = [t['T2'] for t in outcomes]
    round_t1_pct = [t['T1%'] for t in outcomes]
    round_t2_pct = [t['T2%'] for t in outcomes]
    round_analysis = [t['Analysis'] for t in outcomes]
    round_winner = [t['W'] for t in outcomes]
    return round_t1, round_t2, round_t1_pct, round_t2_pct, round_analysis, round_winner


def precomputed_region(roster_list):
    first_round = parse_games(list(precomputed_games_from_list(roster_list)))
    second_round = parse_games(list(precomputed_games_from_list(first_round[0])))
    third_round = parse_games(list(precomputed_games_from_list(second_round[0])))
    fourth_round = parse_games(list(precomputed_games_from_list(third_round[0])))
    return first_round, second_round, third_round, fourth_round

# Making The Spreadsheet

In [ ]:
import shutil

new_bracket = OUTPUT / 'Bracket1.xlsx'
shutil.copyfile(BLANK_BRACKET, new_bracket)

WindowsPath('c:/Users/amgal/Documents/Data Science/KingfisherCourt/output/Bracket1.xlsx')

# Game by Game Bracket

In [13]:
# pip install xlwings

In [ ]:
# Openpyxl somehow created weird invalid references. This library seems to fix the issues.
import xlwings as xw

In [18]:
team_1_col = []
team_2_col = []
scores_1_col = []
scores_2_col = []
analysis_col = []

region_winners = {}
region_data_cache = {}
region_names = ['South', 'West', 'East', 'Midwest']
region_dicts = [south, west, east, midwest]


# --- Rounds of 64 through Elite Eight ---
for region_dict, region_name in zip(region_dicts, region_names):
    computed_data = precomputed_region(list(region_dict.values()))
    region_data_cache[region_name] = computed_data
    region_winners[region_name] = [b[5] for b in computed_data]


for round_idx in range(4):
    for region_name in region_names:
        b = region_data_cache[region_name][round_idx]
        
        team_1_col += b[0]
        team_2_col += b[1]
        scores_1_col += b[2]
        scores_2_col += b[3]
        analysis_col += b[4]

# Final Four
south_west = precomputed_games(region_winners['South'][3][-1], region_winners['West'][3][-1])
east_midwest = precomputed_games(region_winners['East'][3][-1], region_winners['Midwest'][3][-1])

team_1_col += [south_west['T1'], east_midwest['T1']]
team_2_col += [south_west['T2'], east_midwest['T2']]
scores_1_col += [south_west['T1%'], east_midwest['T1%']]
scores_2_col += [south_west['T2%'], east_midwest['T2%']]
analysis_col += [south_west['Analysis'], east_midwest['Analysis']]

# Championship
final = precomputed_games(south_west['W'], east_midwest['W'])

team_1_col.append(final['T1'])
team_2_col.append(final['T2'])
scores_1_col.append(final['T1%'])
scores_2_col.append(final['T2%'])
analysis_col.append(final['Analysis'])

# Excel
wb = xw.Book(new_bracket)
ws = wb.sheets['Games']

for i, row in enumerate(ROW_NUMS):
    ws[f"{TEAM_1_COL}{row}"].value = team_1_col[i]
    ws[f"{TEAM_2_COL}{row}"].value = team_2_col[i]
    ws[f"{SCORE_1_COL}{row}"].value = scores_1_col[i]
    ws[f"{SCORE_2_COL}{row}"].value = scores_2_col[i]
    ws[f"{EVAL_COL}{row}"].value = analysis_col[i]

wb.save(new_bracket)
wb.close()